<a href="https://colab.research.google.com/github/EmanVin/Scratch/blob/sandbox/CategoricalUtilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import scipy.stats as ss
import numpy as np
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x,y)
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    print(chi2)
    n = float(confusion_matrix.sum().sum())
    print(confusion_matrix)
    k = float(min(confusion_matrix.shape)-1)
    return np.sqrt(chi2/n/k)

x = np.array(['a','a','a','a','b','b','b','b','d'])
y = np.array(['y','m','y','x','o','m','w','m','o'])

cramers_v(x,y)

9.374999999999998
col_0  m  o  w  x  y
row_0               
a      1  0  0  1  2
b      2  1  1  0  0
d      0  1  0  0  0


0.7216878364870322

In [133]:
import pandas as pd
import scipy.stats as ss
import numpy as np
from math import inf
import pprint
#gini gain.. closer to 1 means higher gains
def gini_statistic(df, attribute, target):
  values = get_breakdown(df, attribute,target).astype(float).values
  return gini_score(values)
 
def gini_score(values):
  return((values/values.sum())**2).sum()

def get_breakdown(df, attribute, target):
  breakdown = pd.concat([df[attribute],df[target].astype(float)], axis = 1).groupby(attribute).sum()
  return breakdown

  

def get_correl(values):

   return np.corrcoef(values[:,0],values[:,1])



def get_scores(df, attributes, target):
  scores = np.zeros(shape = len(attributes),dtype=float)
  for i in range(len(attributes)):
    scores[i] = gini_statistic(df, attributes[i],target)
    

  return scores,scores.argsort()


result = None
def get_split_tree(df,target,max_depth=inf,depth=0,min_samples = 2,min_sum = 0, allow_residual_split=False):
  indent = "- "*depth
  print("{}total:{}".format(indent,df.shape[0]))
  attributes = df.columns.tolist()
  attributes.remove(target)
  

  (scores,order) = get_scores(df,attributes,target)
  winning_index = np.argmax(scores)

  winning_attribute = attributes[winning_index]
  split_items = df[winning_attribute].unique()
  tree = {'total':df.shape[0]}
          
  result = pd.DataFrame([])
  children_df = pd.DataFrame([])
  
  children = {}  
  split_group = df.groupby(winning_attribute)
  child_attributes = df.columns.tolist()
  child_attributes.remove(winning_attribute)

  total_non_leaf = 0
  residual_split = []
  if split_group.size().min() >= min_samples or allow_residual_split:
    for split_item in split_items:
      sub_df = split_group.get_group(split_item)
      child = {}
      child_df = pd.DataFrame([])
    
      if df.shape[1] >=2 and depth < max_depth and sub_df.shape[0] > min_samples:
        print("{}{}={}".format(indent+"- ",winning_attribute,split_item))
        child_df = get_split_tree(sub_df[child_attributes],target,max_depth = max_depth,depth = depth+1,min_samples= min_samples)
      
        total_non_leaf += sub_df.shape[0]
   
        child_df[depth+1] = "{}-{}:{}".format(winning_attribute,split_item,sub_df.shape[0])  
      else:      
        residual_split.append(split_item)
      
      children_df = pd.concat([children_df,child_df],axis=0)
    if len(residual_split)>0:
      count = df.shape[0]-total_non_leaf
      child_df = pd.DataFrame([count],columns = ["count"])
      child_df[depth+1] = "{}-{}:".format(winning_attribute,"others",count)        
    
      children_df = pd.concat([children_df,child_df],axis=0)
    
  if children_df.shape[0]>0:
      
    print("{}split at:{}".format(indent,winning_attribute))      
    result = children_df
  else:
      
    result['count'] = [df.shape[0]]
  if depth == 0:
    result[0] = 'total:{}'.format(df.shape[0])
  return result

(df,target) = (f1,'price')

tree = get_split_tree(df,target,min_samples=5,allow_residual_split = True)
result = tree

pprint.pprint(result)
print(result["count"].sum())

total:150
- heat=4
- total:105
- - caffiene=3
- - total:45
- - - flavor=2
- - - total:25
- - - - size=0
- - - - total:7
- - - - size=3
- - - - total:6
- - - - size=1
- - - - total:6
- - - - size=2
- - - - total:6
- - - split at:size
- - - flavor=1
- - - total:20
- - split at:flavor
- - caffiene=1
- - total:60
- - - flavor=1
- - - total:36
- - - - size=1
- - - - total:10
- - - - size=3
- - - - total:9
- - - - size=0
- - - - total:11
- - - - size=2
- - - - total:6
- - - split at:size
- - - flavor=2
- - - total:24
- - split at:flavor
- split at:caffiene
- heat=3
- total:30
- - flavor=2
- - total:16
- - - caffiene=3
- - - total:9
- - - caffiene=2
- - - total:7
- - split at:caffiene
- - flavor=1
- - total:14
- - - caffiene=3
- - - total:6
- - - caffiene=2
- - - total:8
- - split at:caffiene
- split at:flavor
- heat=2
- total:15
- - caffiene=2
- - total:15
- - - flavor=1
- - - total:7
- - - flavor=2
- - - total:8
- - split at:flavor
- split at:caffiene
split at:heat
   count          4      

In [132]:
import numpy as np
import pandas as pd
sample = np.random.choice(5,(7,4))

p_groupings = [.3,.4,.1,.1,.1]
total = 150

p_counts = np.round(np.array(p_groupings)*total).astype(int)
seed = [[[4,3]],[[4,1]],[[3,3]],[[2,2]],[[3,2]]]
base = [p for seed_group in [seed[i]*p_counts[i] for i in range(len(seed))] for p in seed_group]
base_array = np.array(base)
column_names = ["flavor","size","heat","caffiene","variant","price"]

f1 = pd.DataFrame(np.concatenate(
     [np.random.choice((1,2),(total,1)).astype(str),
      np.random.choice(4,(total,1)).astype(str),
      base_array.astype(str),
      np.random.choice(6,(total,1)).astype(str),
      np.random.choice(2000,(total,1)).astype(float)

     ],axis=1), columns = column_names
     )

print(f1.shape)
print(f1)

print(f1.groupby(['heat','caffiene']).size())

(150, 6)
    flavor size heat caffiene variant   price
0        2    0    4        3       1  1866.0
1        1    2    4        3       0  1553.0
2        2    3    4        3       0  1572.0
3        1    2    4        3       0   984.0
4        2    0    4        3       4  1340.0
..     ...  ...  ...      ...     ...     ...
145      2    3    3        2       4  1763.0
146      1    2    3        2       5    10.0
147      1    0    3        2       2  1135.0
148      2    3    3        2       4   636.0
149      2    3    3        2       0  1004.0

[150 rows x 6 columns]
heat  caffiene
2     2           15
3     2           15
      3           15
4     1           60
      3           45
dtype: int64


In [122]:

import plotly.express as px
import pandas as pd
  
df = tree
fig = px.sunburst(df.fillna(""), path=list(range(df.shape[1]-1)), values='count')
fig.show()

In [135]:

df = tree

fig = px.treemap(df, path=list(range(df.shape[1]-1)), values='count'
 ,height = 300
,width = 400
)
fig.show()